In [1]:
from brickmapper import Mapper
from rdflib import BRICK
import urllib.request
import csv



In [2]:
# read haystack proto definitions off their website

defns = []
request = urllib.request.Request("https://project-haystack.org/download/protos.csv", headers={'User-Agent': "Python script"})
response = urllib.request.urlopen(request)
rdr = csv.DictReader(response.read().decode('utf8').splitlines())
for row in rdr:
    # save each proto (e.g. 'air temp sensor point') as a dictionary
    defns.append({'name': row['proto']})

defns

[{'name': 'air co2 concentration sensor point'},
 {'name': 'air co2 concentration sp point'},
 {'name': 'air dewPoint sensor point'},
 {'name': 'air dewPoint sp point'},
 {'name': 'air enthalpy sensor point'},
 {'name': 'air enthalpy sp point'},
 {'name': 'air flow sensor point'},
 {'name': 'air flow sp point'},
 {'name': 'air humidity sensor point'},
 {'name': 'air humidity sp point'},
 {'name': 'air pressure sensor point'},
 {'name': 'air pressure sp point'},
 {'name': 'air temp sensor point'},
 {'name': 'air temp sp point'},
 {'name': 'air velocity sensor point'},
 {'name': 'air velocity sp point'},
 {'name': 'alarm sensor point'},
 {'name': 'avg ac elec current angle sensor point'},
 {'name': 'avg ac elec current imbalance sensor point'},
 {'name': 'avg ac elec current magnitude sensor point'},
 {'name': 'avg ac elec current thd sensor point'},
 {'name': 'avg ac elec pf sensor point'},
 {'name': 'avg ac elec volt angle sensor point'},
 {'name': 'avg ac elec volt imbalance sensor po

In [3]:
# create a new Mapper object which knows about the definitions. This will
# access OpenAI to compute embeddings if necessary
m = Mapper(defns, "haystack_proto.index")

# define which set of Brick classes we want to match against
# all points which aren't parameters (these have similar enough names it can confuse the mapping)
points = m.get_brick_classes(BRICK.Point, [BRICK.Parameter])
# all equipment in Brick
equips = m.get_brick_classes(BRICK.Equipment)
# concatenate these two lists of Brick classes
bcs = points + equips

bcs

2024-12-19 13:29:37.159 | INFO     | brickmapper:populate_external_embeddings:66 - Restored haystack_proto.index


[BrickClassDefinition(class_=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Static_Pressure_Sensor'), label='Static Pressure Sensor', definition="Measures resistance to airflow in a heating and cooling system's components and duct work", definitions=[]),
 BrickClassDefinition(class_=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Underfloor_Air_Plenum_Static_Pressure_Sensor'), label='Underfloor Air Plenum Static Pressure Sensor', definition='Measures the outward push of air against the plenum surfaces and used to measure the resistance when air moves through the plenum', definitions=[]),
 BrickClassDefinition(class_=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Air_Quality_Sensor'), label='Air Quality Sensor', definition='A sensor which provides a measure of air quality', definitions=[]),
 BrickClassDefinition(class_=rdflib.term.URIRef('https://brickschema.org/schema/Brick#Entering_Medium_Temperature_Hot_Water_Temperature_Load_Shed_Setpoint'), label='Enter

In [4]:
# compute a 'stable' mapping (1:1 mapping)
stable = m.get_mapping(bcs, allow_collisions=False)
# compute an 'unstable' mapping that allows 1:n and n:1 matches
unstable = m.get_mapping(bcs, allow_collisions=True)

# loop through and print the stable/unstable matches
for k,v in unstable.items():
    print(f"{k}:\n\tunstable: {v}\n\tstable: {stable.get(k)}")


00%|████████████████████████████████████████████████████████████████████████████████████| 1121/1121 [00:00<00:00, 2830.12it/s]

{rdflib.term.URIRef('https://brickschema.org/schema/Brick#Absolute_Humidity_Sensor'): ['outside '
                                                                                       'air '
                                                                                       'humidity '
                                                                                       'sensor '
                                                                                       'point',
                                                                                       'ventilation '
                                                                                       'air '
                                                                                       'humidity '
                                                                                       'sensor '
                                                                                       'point',
                            

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

